In [1]:
import arpeggio

In [2]:
from csv import DictReader

In [51]:
from typing import List
from arpeggio import RegExMatch as _
from arpeggio import RegExMatch, ParserPython, OneOrMore

number =  _(r'\d*\.\d*|\d+')
from arpeggio import Optional

meters = _(r"m|metres?|meters?")
kilometers = _(r"km|kilometres?|kilometers?")
miles = _(r"mi|miles?")

distance_units =  [miles, meters, kilometers]

feature_type = ["restaurant", "bar"]

token = _(r"\S+")
toponymn = OneOrMore(token)
location = [feature_type, toponymn]

sprel_in = "in"
sprel_fuzzy_dist = (Optional("very"), ["near", "far"])

sprel_in_distance = (["within", "in"], number, distance_units, Optional("of"))
sprel__distance = (["within", "in"], number, distance_units, Optional("of"))

spatial_relation = [sprel_in_distance, sprel_in]

grammar = (feature_type, spatial_relation, location)

parser = ParserPython(grammar)

parser.parse("restaurant within 5 km of Oakland")

[[ 'restaurant' [0]],
 [[[ 'within' [11]],  '5' [18], [ 'km' [20]], [ 'of' [23]]]],
 [[ 'Oakland' [26]]]]

In [6]:
import pandas as pd
df = pd.read_csv("../data/nominatim-special-phrases.csv")

In [66]:
import json
spatial_feature_type_lexicon = df.loc[df["Operator"] == "-", :]["Word / Phrase"].str.lower().to_list()
with open("../data/spatial_feature_types_lexicon.json", "w") as f:
    json.dump(spatial_feature_type_lexicon, f)

In [12]:
import json
spatial_feature_type_lexicon = df.loc[df["Operator"] == "-", :]
words = {}
for r in spatial_feature_type_lexicon.to_dict(orient="records"):
    words[r["Word / Phrase"].lower()] = {"key": r["Key"], "value": r["Value"], "plural": r["Plural"] == "Y"}
with open("../data/spatial_feature_types_lexicon.json", "w") as f:
    json.dump(words, f)  


in 

```
FROM A
INNER JOIN B 
    ON ST_Within(A, B)
    AND B IS Area
```

near 

```
FROM A
INNER JOIN B
   ON ST_DWithin(A, B, :near_d)
```

far 
```
FROM A
INNER JOIN B
   ON NOT ST_DWithin(A, B, :far_d)
```

crosses
```
FROM A
INNER JOIN B
   ON NOT ST_Intersects(A, B, :far_d)
   AND A IS line
```

borders
```
FROM A
INNER JOIN B
   ON NOT ST_Touches(A, B, :far_d)
```


In [71]:
import wn